In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv, find_dotenv
import os
import matplotlib.pyplot as plt
import re
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import Normalizer

In [2]:
df_goalkeeper = df.copy()

def clean_gk_column(string):
    html_string = '<span class="pos pos16">LM'

    # Define a regular expression pattern to match HTML tags
    html_tags_pattern = re.compile(r'<.*?>')

    # Use the sub() function to replace matched HTML tags with an empty string
    cleaned_string = html_tags_pattern.sub('', str(string))
    return cleaned_string

# clean Position column
df_goalkeeper['Position'] = df_goalkeeper['Position'].apply(clean_gk_column)
# select only goal keeper
df_goalkeeper = df_goalkeeper[df_goalkeeper['Position'] == 'GK']

# remove columns:
columns_to_keep_gk = ['Age','GKDiving', 'GKKicking', 'GKPositioning', 'GKReflexes','Reactions', 'Vision', 'Composure','LongPassing', 'Strength','Jumping', 'Overall']
df_goalkeeper = df_goalkeeper[columns_to_keep_gk]


# handle naivly NaN's
df_goalkeeper.loc[df_goalkeeper['Composure'].isna(), 'Composure'] = df_goalkeeper['Composure'].median()
df_goalkeeper.loc[df_goalkeeper['Jumping'].isna(), 'Jumping'] = df_goalkeeper['Jumping'].median()
df_goalkeeper.loc[df_goalkeeper['Vision'].isna(), 'Vision'] = df_goalkeeper['Vision'].median()

# store data frame
# df_goalkeeper.to_csv('../data/df_goalkeeper.csv', sep=',')
display(df_goalkeeper)

NameError: name 'df' is not defined

In [ ]:
corr_matrix = df_goalkeeper.corr()
plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
df_goalkeeper_min_max = df_goalkeeper.copy()
# Min-max normalization function
def min_max_normalize(column):
    return (column - column.min()) / (column.max() - column.min())

columns_to_normalize = ['GKDiving','GKKicking','GKPositioning', 'GKReflexes','Reactions','Vision','Composure', 'LongPassing','Strength','Jumping','Overall']
df_goalkeeper_min_max[columns_to_normalize] = df_goalkeeper_min_max[columns_to_normalize].apply(min_max_normalize)

display(df_goalkeeper_min_max['Overall'])


In [ ]:
y = df_goalkeeper['Overall']
X = df_goalkeeper.drop('Overall', axis=1)

# Assuming X contains your features and y contains your target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Scale data X
scaler = StandardScaler()
columns_to_normalize = ['GKDiving','GKKicking','GKPositioning', 'GKReflexes','Reactions','Vision','Composure', 'LongPassing','Strength','Jumping','Overall']
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# df_goalkeeper[columns_to_normalize] = scaler.fit_transform(df_goalkeeper[columns_to_normalize])
display(X_train[0])

In [ ]:
# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error as an example evaluation metric
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# One-hot encode the target variable
one_hot_encoder = OneHotEncoder(sparse=False, categories='auto')
y_one_hot = one_hot_encoder.fit_transform(y.reshape(-1, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Reshape the data for LSTM input (samples, time steps, features)
time_steps = 1  # We treat each sample as a single time step
X_train_lstm = X_train.reshape(X_train.shape[0], time_steps, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], time_steps, X_test.shape[1])

# Build the LSTM model
model = Sequential()
model.add(LSTM(10, input_shape=(time_steps, X_train.shape[1])))
model.add(Dense(3, activation='softmax'))  # Output layer with 3 classes

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_lstm, y_train, epochs=50, batch_size=8, validation_split=0.1)

# Evaluate the model on the test set
accuracy = model.evaluate(X_test_lstm, y_test)[1]
print("Test Accuracy:", accuracy)
